<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/text_generation_pipeline_conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation example with conversational model and model quantization

This is a brief example of how to run text generation with a conversational (chat, instruct) causal language model, `pipeline`, and model quantization.

[Model quantization](https://huggingface.co/docs/accelerate/usage_guides/quantization) is a technique to reduce the computational and memory costs of running inference by representing the weights and activations with low-precision data types like 8-bit integer (int8) instead of the usual 32-bit floating point (float32).

In [ ]:
!pip install --quiet transformers accelerate bitsandbytes>0.37.0

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience. `BitsAndBytesConfig` is used for model quantization.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

Load a generative model and its tokenizer. You can substitute any other generative model name here, but note that Colab may have issues running larger models.

`Mistral-7B-Instruct-v0.2` model needs authentication, `userdata.get('my_access_token')` reads my secret authentication token I created for my hf account.

In [ ]:
from google.colab import userdata
import torch

MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side='left', token=userdata.get('my_access_token'))
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, token=userdata.get('my_access_token'), device_map="auto", quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Instantiate a text generation pipeline using the tokenizer and model.

In [ ]:
from transformers import Conversation

pipe = pipeline("conversational",
    model=model,
    tokenizer=tokenizer
)

We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [ ]:
conversation = Conversation("What is the capital of Finland?")
output = pipe(conversation, max_new_tokens=50)

print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Conversation id: d6da97d5-2b4e-4d76-a534-8ef1d0dd5971
user: What is the capital of Finland?
assistant: The capital city of Finland is Helsinki. It is the largest city in Finland and is located in the southern part of the country, on the shores of the Gulf of Finland. Helsinki is known for its beautiful architecture,



Just print the text

We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [ ]:
conversation = Conversation("What is the capital of Finland?")
output = pipe(conversation,
              do_sample=True,
              temperature=3.0,
              max_new_tokens=50)

print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Conversation id: 46135045-a5a1-4abb-b256-5647a18d61a6
user: What is the capital of Finland?
assistant: The cultural and administrative heart of Finland is defined by its enchanting capital city:- Helsинki. (English: Helsinki) Positioned elegantly where the clear waters met the cerullean seas from the Bay of Both Nina

